# Foundation Models para Dados Tabulares
## Comparacao: XGBoost vs TabPFN vs MITRA

Este notebook compara modelos classicos (XGBoost) com **Foundation Models** para dados tabulares.

### O que sao Foundation Models Tabulares?

| Modelo | Descricao |
|--------|----------|
| **TabPFN** | Transformer pre-treinado em datasets sinteticos |
| **MITRA** | Mixed Synthetic Priors - state-of-the-art da AutoGluon |

### Trade-off: Performance vs Interpretabilidade

| Modelo | Performance | Interpretabilidade |
|--------|-------------|-------------------|
| XGBoost + SHAP | Boa | **Excelente** |
| TabPFN/MITRA | **Excelente** | Limitada |

> **Importante**: Em credit scoring, explicabilidade e requisito regulatorio (LGPD, BACEN).

---
## 1. Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

from credit_scoring.data.loader import load_german_credit
from credit_scoring.models.train import load_model

print("Setup completo!")

In [ ]:
# Carregar dados
X, y = load_german_credit(save_raw=False)

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Treino: {len(X_train)} | Teste: {len(X_test)}")

---
## 2. Baseline: XGBoost (ja treinado)

In [ ]:
# Carregar modelo XGBoost otimizado
xgb_model = load_model('../models/best_model_optuna.joblib')

# Predicoes
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
xgb_pred = xgb_model.predict(X_test)

# Metricas
xgb_metrics = {
    'Model': 'XGBoost (Optuna)',
    'ROC-AUC': roc_auc_score(y_test, xgb_proba),
    'F1-Score': f1_score(y_test, xgb_pred),
    'Precision': precision_score(y_test, xgb_pred),
    'Recall': recall_score(y_test, xgb_pred),
    'Interpretable': 'Sim (SHAP)'
}

print(f"XGBoost ROC-AUC: {xgb_metrics['ROC-AUC']:.4f}")

---
## 3. TabPFN

TabPFN e um transformer pre-treinado que faz predicao sem necessidade de treino.

In [ ]:
# Instalacao (se necessario)
# !pip install tabpfn

try:
    from tabpfn import TabPFNClassifier
    HAS_TABPFN = True
    print("TabPFN disponivel!")
except ImportError:
    HAS_TABPFN = False
    print("TabPFN nao instalado. Execute: pip install tabpfn")

In [ ]:
tabpfn_metrics = None

if HAS_TABPFN:
    # Preprocessar dados para TabPFN (apenas valores numericos)
    X_train_processed = xgb_model.named_steps['preprocessor'].transform(X_train)
    X_test_processed = xgb_model.named_steps['preprocessor'].transform(X_test)
    
    # TabPFN
    tabpfn = TabPFNClassifier(device='cpu', N_ensemble_configurations=16)
    tabpfn.fit(X_train_processed, y_train)
    
    tabpfn_proba = tabpfn.predict_proba(X_test_processed)[:, 1]
    tabpfn_pred = tabpfn.predict(X_test_processed)
    
    tabpfn_metrics = {
        'Model': 'TabPFN',
        'ROC-AUC': roc_auc_score(y_test, tabpfn_proba),
        'F1-Score': f1_score(y_test, tabpfn_pred),
        'Precision': precision_score(y_test, tabpfn_pred),
        'Recall': recall_score(y_test, tabpfn_pred),
        'Interpretable': 'Limitada'
    }
    
    print(f"TabPFN ROC-AUC: {tabpfn_metrics['ROC-AUC']:.4f}")
else:
    print("Pulando TabPFN (nao instalado)")

---
## 4. MITRA (AutoGluon)

MITRA e o state-of-the-art em foundation models tabulares, desenvolvido pela AWS/AutoGluon.

In [ ]:
# Instalacao (se necessario)
# !pip install autogluon.tabular[mitra]

try:
    from autogluon.tabular import TabularPredictor
    HAS_AUTOGLUON = True
    print("AutoGluon disponivel!")
except ImportError:
    HAS_AUTOGLUON = False
    print("AutoGluon nao instalado. Execute: pip install autogluon.tabular[mitra]")

In [ ]:
mitra_metrics = None

if HAS_AUTOGLUON:
    # Preparar dados
    train_data = X_train.copy()
    train_data['target'] = y_train.values
    
    test_data = X_test.copy()
    
    # Treinar MITRA
    predictor = TabularPredictor(
        label='target',
        eval_metric='roc_auc',
        path='../models/mitra_model'
    ).fit(
        train_data,
        hyperparameters={'MITRA': {}},
        time_limit=60,
        verbosity=1
    )
    
    mitra_proba = predictor.predict_proba(test_data).values[:, 1]
    mitra_pred = predictor.predict(test_data).values
    
    mitra_metrics = {
        'Model': 'MITRA (AutoGluon)',
        'ROC-AUC': roc_auc_score(y_test, mitra_proba),
        'F1-Score': f1_score(y_test, mitra_pred),
        'Precision': precision_score(y_test, mitra_pred),
        'Recall': recall_score(y_test, mitra_pred),
        'Interpretable': 'Limitada'
    }
    
    print(f"MITRA ROC-AUC: {mitra_metrics['ROC-AUC']:.4f}")
else:
    print("Pulando MITRA (AutoGluon nao instalado)")

---
## 5. Comparacao Final

In [ ]:
# Consolidar resultados
results = [xgb_metrics]

if tabpfn_metrics:
    results.append(tabpfn_metrics)
if mitra_metrics:
    results.append(mitra_metrics)

comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 70)
print("COMPARACAO: XGBoost vs Foundation Models")
print("=" * 70)
print(comparison_df.to_string(index=False))

In [ ]:
# Visualizacao
fig, ax = plt.subplots(figsize=(10, 6))

models = comparison_df['Model'].tolist()
auc_scores = comparison_df['ROC-AUC'].tolist()

colors = ['#2ecc71' if 'XGBoost' in m else '#3498db' for m in models]
bars = ax.barh(models, auc_scores, color=colors, edgecolor='black')

# Adicionar valores
for bar, score in zip(bars, auc_scores):
    ax.text(score + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{score:.4f}', va='center', fontweight='bold')

ax.set_xlabel('ROC-AUC Score', fontsize=12)
ax.set_title('Comparacao: Modelos Classicos vs Foundation Models', fontsize=14, fontweight='bold')
ax.set_xlim(0.7, 0.9)
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
ax.legend()

plt.tight_layout()
plt.savefig('../reports/figures/foundation_models_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Analise de Interpretabilidade

### XGBoost: Interpretabilidade Completa com SHAP
- Explicacoes globais e locais
- Feature importance
- Waterfall plots por cliente

### Foundation Models: Interpretabilidade Limitada
- **Attention weights**: Pode indicar features relevantes, mas nao e intuitivo
- **Permutation importance**: Possivel mas computacionalmente caro
- **SHAP parcial**: Pode ser aplicado mas com limitacoes

In [ ]:
# Permutation Importance para Foundation Models (se disponiveis)
from sklearn.inspection import permutation_importance

if HAS_TABPFN and tabpfn_metrics:
    print("Calculando Permutation Importance para TabPFN...")
    result = permutation_importance(
        tabpfn, X_test_processed, y_test,
        n_repeats=5, random_state=42, scoring='roc_auc'
    )
    
    # Obter nomes das features do preprocessor
    feature_names = xgb_model.named_steps['preprocessor'].get_feature_names_out()
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': result.importances_mean
    }).sort_values('importance', ascending=False).head(10)
    
    print("\nTop 10 Features (Permutation Importance - TabPFN):")
    print(importance_df.to_string(index=False))

---
## 7. Conclusao e Recomendacao

In [ ]:
print("=" * 70)
print("CONCLUSAO")
print("=" * 70)

print("""
PARA PRODUCAO EM CREDIT SCORING:

| Cenario                          | Modelo Recomendado |
|----------------------------------|--------------------|
| Regulatorio exige explicacoes    | XGBoost + SHAP     |
| Prototipo/PoC rapido             | TabPFN/MITRA       |
| Ensemble                         | Combinar ambos     |
| Performance pura                 | MITRA              |

RECOMENDACAO FINAL:
- Usar XGBoost como modelo principal (explicabilidade)
- Usar Foundation Models como benchmark ou ensemble
- Documentar trade-offs para stakeholders
""")

# Salvar resultados
comparison_df.to_csv('../reports/foundation_models_comparison.csv', index=False)
print("\nResultados salvos em: reports/foundation_models_comparison.csv")